# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie III : Data modeling

Ce notebook a pour but de présenter le travail effectué sur la modélisation.

Nous commencerons par séparer notre jeu de données en deux parties distinctes:
- Le **training set**, qui va permettre d'entrainer les différents modèles;
- Le **testing set**, qui permettra de déterminer la performance du modèle finale.

Pour ce faire, la méthode `train_test_split()` de la classe *sklearn.model_selection* sera utilisée en réservant 20% des données pour le jeu de test.

Puis les modèles les plus courants seront entraînés et comparés afin de conserver les plus prometteurs. Au préalable, *une recherche par quadrillage* sera effectuée pour automatiser le choix des *hyperparamètres*, et les variables les plus pertinentes seront sélectionnées par **RFE** (Recursive Feature Elimination).

Après sélection des modèles les plus performants, nous affinerons encore les hyperparamètres à l'aide d'une *recherche aléatoire* cette fois ci, et nous en profiterons pour tester la pertinence de la variable *EnergyStarScore*.

Nous analyserons enfin les erreurs des modèles afin de déterminer s'il est pertinent d'utiliser une *méthode d'ensemble*, ie. combiner plusieurs modèles pour construire un modèle plus performant.

Le modèle final obtenu, nous pourrons évaluer sa performance à l'aide du jeu de test.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

In [3]:
# import data
data = (pd.read_csv('data/data_tr.csv')
          .set_index('OSEBuildingID')
          .drop(columns='ENERGYSTARScore'))
data_ess = (pd.read_csv('data/data_tr.csv')
              .set_index('OSEBuildingID')
              .dropna(subset=['ENERGYSTARScore']))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span><ul class="toc-item"><li><span><a href="#En-conservant-les-valeurs-par-défaut-des-hyperparamères" data-toc-modified-id="En-conservant-les-valeurs-par-défaut-des-hyperparamères-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>En conservant les valeurs par défaut des hyperparamères</a></span></li><li><span><a href="#En-optimisant-les-hyperparamères-via-des-recherches-par-grille-ou-aléatoires" data-toc-modified-id="En-optimisant-les-hyperparamères-via-des-recherches-par-grille-ou-aléatoires-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>En optimisant les hyperparamères via des recherches par grille ou aléatoires</a></span></li></ul></li><li><span><a href="#Recherche-des-variables-les-plus-significatives" data-toc-modified-id="Recherche-des-variables-les-plus-significatives-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Recherche des variables les plus significatives</a></span></li><li><span><a href="#Sélection-des-cinq-meilleurs-modèles" data-toc-modified-id="Sélection-des-cinq-meilleurs-modèles-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Sélection des cinq meilleurs modèles</a></span></li><li><span><a href="#Modèle-final" data-toc-modified-id="Modèle-final-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Modèle final</a></span></li></ul></li></ul></div>

## Modéliser la consommation totale d’énergie

Pour pouvoir modéliser la consommation totale en énergie il faut au préalable déterminer quelle sera notra valeur cible. Sous nous avons quatre variables potentielles :
- SiteEnergyUse(kBtu)
- SiteEnergyUseWN(kBtu)
- SiteEnergyUse(kBtu)_log
- SiteEnergyUseWN(kBtu)_log

Nous avons vu lors de l'ingénierie des variable qu'il était préférable de prendre la version log pour avoir une distribution se rapprochant d'une distribution normale. On peut donc déjà écarter les deux premières.

Se pose ensuite la question de savoir s'il est préférable de garder la version normalisée ou non normalisée. Pour rappel, la version normalisée et la consommation corrigée en prenant comme référence la température des trentes dernières années. Alors que la version normalisée est la consommation moyenne sur les années 2015 et 2016. Dans le contexte de réchauffement climatique, il est fort à parier que la températures des prochaines années sera plus proche de celles de 2015 et 2016 que de la température des trentes dernières années.

Nous prendrons donc la version non normalisée **SiteEnergyUse(kBtu)_log**.

### Créer un jeu de test

In [4]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

Le paramètre *random_state* permet de définir *le germe* (seed) du générateur de nombre aléatoires, afin qu'il génère toujours la même suite d'indices pseudo-aléatoires.

### Comparaison des modèles

Pour comparer les résultats obtenus avec les algorithmes de regression les plus courants (voir liste ci-dessous), nous utiliserons la librairie *Scikit-Learn* ainsi que la librairie *XGBoost*.
- Régression Ridge
- Régression Lasso
- Elastic Net
- Régression SVM linéaire
- Régression SVM avec noyau
- Régression kNN
- Arbre de décision
- Forêt aléatoire
- Gradient Boosting
- XG Boost
- Perceptron multi-couches

Nous utiliserons comme mesure de performance la RMSE par validation croisée.

#### En conservant les valeurs par défaut des hyperparamères

In [5]:
from functions.ml_modeling import get_models
from functions.ml_modeling import compare_models

# Create models
models = get_models(X_train, y_train)

# Compare models
df = compare_models(X_train, y_train, models)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,1.244510,0.751661,-0.207031,1.513799
Lasso,1.302400,0.028977,-0.002535,0.004503
ElasticNet,1.282373,0.036099,0.028226,0.015908
LinearSVR,1.297330,0.764975,-0.298496,1.574040
SVR,0.839410,0.042076,0.582491,0.039502
KNeighborsRegressor,1.002512,0.043741,0.403637,0.061485
DecisionTreeRegressor,0.886254,0.061504,0.534470,0.054831
RandomForestRegressor,0.650826,0.036142,0.749069,0.025033
MLPRegressor,1.976235,1.172932,-2.046453,3.744264
GradientBoostingRegressor,0.635029,0.029781,0.761241,0.020096


#### En optimisant les hyperparamères via des recherches par grille ou aléatoires

In [6]:
# Create models
models = get_models(X_train, y_train, best_hparams=True)

# Compare models
df = compare_models(X_train, y_train, models)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.963020,0.058076,0.450730,0.057112
Lasso,1.150223,0.226098,0.196524,0.318347
ElasticNet,1.017612,0.123332,0.382556,0.140209
LinearSVR,1.223609,0.386000,0.040312,0.637076
SVR,0.785806,0.058818,0.633079,0.053247
KNeighborsRegressor,0.980812,0.038988,0.430517,0.041821
DecisionTreeRegressor,0.741738,0.038038,0.674374,0.027927
RandomForestRegressor,0.645786,0.035395,0.752941,0.024605
MLPRegressor,2.004750,1.266686,-2.236833,4.316351
GradientBoostingRegressor,0.634854,0.029932,0.761374,0.020126


### Recherche des variables les plus significatives

Pour rechercher les variables les plus significatives, nous effectuerons une *RFE* (Recursive Feature Elimination) sur le modèle RandomForest qui permet de mesurer facilement l'importance des variables. Une validation croisée sera effectuée pour connaître quel est le meilleur nombre de variables à conserver.

Ensuite nous ne conserverons que les variables qui sont significatives et réentrainerons les modèles avec ces variables.

In [7]:
models_rfe = [m for m in models if type(m).__name__=='RandomForestRegressor']

# Run the RFE for each features
df = compare_models(X_train, y_train, models_rfe, rfe=True)

# Keep the most relevant features only
mask = df['mask_Features'].values.sum(axis=0)
X_rfe = X_train[:, mask]
n_features = mask.sum()
print("{} variables ont été conservées :".format(n_features))
for c in data.columns[:-6][mask]:
    print(c)

# Train the best models on the filtered data
df = compare_models(X_rfe, y_train, models)
df

54 variables ont été conservées :
Latitude
Longitude
NumberofBuildings
NumberofFloors
PropertyGFATotal
YearBuilt
SteamUse_ratio
Electricity_ratio
NaturalGas_ratio
OtherFuel_ratio
PropertyGFAParking_ratio
PropertyGFABuilding(s)_ratio
LargestPropertyUseTypeGFA_ratio
SecondLargestPropertyUseTypeGFA_ratio
ThirdLargestPropertyUseTypeGFA_ratio
Office
Medical Office
Other
Retail Store
K-12 School
Restaurant/Bar
Hotel
University
Self-Storage Facility
Warehouse
Senior Care Community
Hospital
Residence Hall
Distribution Center
Worship Facility
Supermarket / Grocery Store
Laboratory
Refrigerated Warehouse
Supermarket/Grocery Store
Multifamily Housing
BALLARD
Ballard
CENTRAL
Central
DELRIDGE
DELRIDGE NEIGHBORHOODS
DOWNTOWN
Delridge
EAST
GREATER DUWAMISH
LAKE UNION
MAGNOLIA / QUEEN ANNE
NORTH
NORTHEAST
NORTHWEST
North
Northwest
SOUTHEAST
SOUTHWEST


,RMSE,RMSE_std,R2,R2_std
Ridge,0.963020,0.058076,0.450730,0.057112
Lasso,1.150223,0.226098,0.196524,0.318347
ElasticNet,1.017612,0.123332,0.382556,0.140209
LinearSVR,1.223609,0.386000,0.040312,0.637076
SVR,0.785806,0.058818,0.633079,0.053247
KNeighborsRegressor,0.980812,0.038988,0.430517,0.041821
DecisionTreeRegressor,0.741738,0.038038,0.674374,0.027927
RandomForestRegressor,0.645786,0.035395,0.752941,0.024605
MLPRegressor,1.977667,1.224102,-2.114932,3.950287
GradientBoostingRegressor,0.634710,0.029950,0.761481,0.020176


### Sélection des cinq meilleurs modèles

In [8]:
# Get a score based on both RMSE and R2 scores
df['RMSE_minmax'] = (df['RMSE']-df['RMSE'].min())/(df['RMSE'].max()-df['RMSE'].min())
df['R2_minmax'] = (df['R2']-df['R2'].min())/(df['R2'].max()-df['R2'].min())
df['Score'] = df['R2_minmax'] - df['RMSE_minmax']

# Keep the 3 models with the largest score
best_models_name = df['Score'].nlargest(5).index
best_models = [m for m in models if type(m).__name__ in best_models_name]
print("The top three models are:")
for m in best_models_name:
    print(m)

The top three models are:
GradientBoostingRegressor
RandomForestRegressor
XGBRegressor
DecisionTreeRegressor
SVR


### Modèle final

Maintenant que nous avons réentrainé les modèles les plus prometteurs en ne conservant que les variables significatives, nous allons pouvoir obtenir le modèle finale.

Pour ce modèle finale, nous allons reprendre l'astuce des modèles ensemblistes, nous allons chercher à obtenir le meilleur des modèles sélectionnés en les empilant de manière à former un nouveau modèle plus performant. Pour cela, nous utiliserons la classe **StackingRegressor** de *sklearn.ensemble*.

In [9]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

# Get a stacking ensemble of models
estimators = []
for m in best_models:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))

# Get the results
best_models.append(stack_reg)
df = compare_models(X_rfe, y_train, best_models)
df

,RMSE,RMSE_std,R2,R2_std
SVR,0.785806,0.058818,0.633079,0.053247
DecisionTreeRegressor,0.741738,0.038038,0.674374,0.027927
RandomForestRegressor,0.645786,0.035395,0.752941,0.024605
GradientBoostingRegressor,0.634842,0.030437,0.761385,0.020345
XGBRegressor,0.659871,0.030612,0.742180,0.021363
StackingRegressor,0.625105,0.034127,0.768456,0.023290


In [10]:
models_stacking = models[:1] + models[4:6] + models[7:]

# Get a stacking ensemble of models
estimators = []
for m in models_stacking:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))

# Get the results
models_stacking.append(stack_reg)
df = compare_models(X_rfe, y_train, models_stacking)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.963020,0.058076,0.450730,0.057112
SVR,0.785806,0.058818,0.633079,0.053247
KNeighborsRegressor,0.980812,0.038988,0.430517,0.041821
RandomForestRegressor,0.645786,0.035395,0.752941,0.024605
MLPRegressor,1.920529,1.155635,-1.888528,3.541454
GradientBoostingRegressor,0.634594,0.030372,0.761572,0.020279
XGBRegressor,0.659871,0.030612,0.742180,0.021363
StackingRegressor,0.619733,0.034476,0.772194,0.025441
